In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops sentencepiece bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.0 MB/s eta 0:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login


In [ ]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
    # Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()


CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: Tesla T4
Compute Capability: (7, 5)
Total Memory: 15835660288 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.1.58+
Python Version: 3.10.12


In [ ]:
notebook_login()

In [ ]:
# Pre trained model
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Hugging face repository link to save fine-tuned model
new_model = "TymofiiNas/test-mistral"

In [ ]:
# Downloading dataset
dataset = load_dataset("csv",data_files="data-train-llm2.csv", split="train")
dataset["caption"][0]


Generating train split: 0 examples [00:00, ? examples/s]

'While navigating a myriad of life challenges, Anjelica Avella turned to the outdoors to improve her mental health only to discover a noticeable lack of other plus-size women of color on trail. By posting a simple flyer looking for more women to hike with, Anjelica created a community she never could have imagined.\n\nThis International Women’s Day, @smartwool and @ospreypacks are thrilled to present Forward, a new short documentary sharing the life and perspective of plus-size hiker @adventureswithanjelica produced by @sprucetonefilms.\n\nLearn more and watch now via the link in our stories and bio! English CC, Spanish subtitles, and English Audio Description are available.\n\nAdditional support from @eddiebauer @wildernesssociety and @lekiusa. \n\n#smartwool #osprey\xa0 #gofarfeelgood #internationalwomensday #diversifyoutdoors #latinooutdoors #outdoorsforall #representationmatters'

In [ ]:
# Load base model and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.config.pretraining_tp = 1
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [ ]:
#monitering login
wandb.login(key="<key>")
run = wandb.init(project='Fine tuning mistral-2-7B', job_type="training", anonymous="allow")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: leaster1337. Use `wandb login --relogin` to force relogin


In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [ ]:
peft_config = LoraConfig(
    lora_alpha= 16,
    lora_dropout= 0.1,
    r= 8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj", "up_proj"]
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 5,
    per_device_train_batch_size= 2,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb")

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    text = f"<s>[INST]{example['answer']}[/INST] {example['caption']}</s>"
    output_texts.append(text)
    return output_texts

In [ ]:
print(formatting_prompts_func(dataset[0]))

['<s>[INST]write caption for a documentary about plus-size women hikers[/INST] While navigating a myriad of life challenges, Anjelica Avella turned to the outdoors to improve her mental health only to discover a noticeable lack of other plus-size women of color on trail. By posting a simple flyer looking for more women to hike with, Anjelica created a community she never could have imagined.\n\nThis International Women’s Day, @smartwool and @ospreypacks are thrilled to present Forward, a new short documentary sharing the life and perspective of plus-size hiker @adventureswithanjelica produced by @sprucetonefilms.\n\nLearn more and watch now via the link in our stories and bio! English CC, Spanish subtitles, and English Audio Description are available.\n\nAdditional support from @eddiebauer @wildernesssociety and @lekiusa. \n\n#smartwool #osprey\xa0 #gofarfeelgood #internationalwomensday #diversifyoutdoors #latinooutdoors #outdoorsforall #representationmatters</s>']


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    formatting_func=formatting_prompts_func,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/486 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=5, training_loss=0.8305186271667481, metrics={'train_runtime': 63.5754, 'train_samples_per_second': 0.079, 'train_steps_per_second': 0.079, 'total_flos': 218938768097280.0, 'train_loss': 0.8305186271667481, 'epoch': 5.0})

In [ ]:
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

train/epoch,▁
train/global_step,▁
total_flos,218938768097280.0
train/epoch,5.0
train/global_step,5
train_loss,0.83052
train_runtime,63.5754
train_samples_per_second,0.079
train_steps_per_second,0.079


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=

In [ ]:
trainer.push_to_hub(new_model)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/65.1M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TymofiiNas/results/commit/4034641188f8f7b4a2cbc545caf91d6ec2cd489a', commit_message='TymofiiNas/test-mistral', commit_description='', oid='4034641188f8f7b4a2cbc545caf91d6ec2cd489a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
merged_model = trainer.model.merge_and_unload()

NameError: name 'trainer' is not defined

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=False)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=400, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [ ]:
generate_response("<s>[INST] write a instagram post about my old backpack  [/INST]", trainer.model)

'<s> [INST] write a instagram post about my old backpack  [/INST] 📸: "#ThrowbackThursday to my trusty old backpack ���buchbag���::_ I\'ve traveled the world with this reliable companion 🌍, carried all my essentials from college classes to international flights, and even served as a makeshift pillow on long bus rides. 🛫 Every scratch, every stain tells a story of adventure and memories made. This backpack may not be the sleekest or newest model, but it holds a special place in my heart and has earned its spot as a reliable travel buddy for life. #VintageFind #OldButGold #TravelStories #BackpackAdventures #FlashbackFriday" 📷: [Insert your old backpack picture] #Hashtags: #adventuretime #backpacking #memories #worldtravel #explore #rememberwhen #throwback #journey #travelmore #wanderlust #exploremore #backpackerlife #backpackersofinstagram #backpackingcommunity #exploretocreate #documentyourjourney #backpackadventures #travelingonabudget #theworldisourplayground</s>'